![ML Logo](https://raw.githubusercontent.com/chicochica10/utad-spark-ml/master/images/utad-spark-ml.1x_Banner_300.png)
# ** Lab de Análisis de componentes principales**
#### Este lab se adentra en el análisis exploratorio de datos del mundo de la neurociencia (no os asustéis, es más fácil de lo que parece) en concreto vamos a utilizar el análisis de componenentes principales (PCA) y la agregación basada en características. Los datos que usaremos son del laboratorio  [Ahrens](http://www.janelia.org/lab/ahrens-lab) y están almacenados en el siguiente [data repository](http://datasets.codeneuro.org).

#### Los datos provienen del estudio de un pez transparente llamado [pez cebra](http://en.wikipedia.org/wiki/Zebrafish) al ser transparente es posible registrar su actividad neuronal aplicando la técnica [light-sheet microscopy](http://en.wikipedia.org/wiki/Light_sheet_fluorescence_microscopy).  En concreto vamos a trabajar con los datos neuronales del pez cebra al ser expuesto a distintos estímulos visuales móviles. Éstos estímulos inducen diferentes patrones en el cerebro y podemos utilizar análisis exploratorio para identificarlos.  Más información en  ["Mapping brain activity at scale with cluster computing"](http://thefreemanlab.com/work/papers/freeman-2014-nature-methods.pdf) 

#### En ete lab aprenderás a trabajar con PCA y comparar diferentes análisis exploratorios sobre los mismos datos para identificar que patrones neuronales son los que más resaltan.

#### ** Este lab cubre: **
+  ####*Parte 1:* Pasos de un PCA sobre un dataset de muestra
 + ####*Visualización 1:* Gausianas de dos dimensiones
+  ####*Parte 2:* Escribir una función PCA y evaluación en el dataset de muestra
 + ####*Visualización 2:* Proyección PCA
 + ####*Visualización 3:* Datos tridimensionales
 + ####*Visualización 4:* Respresentación en 2D de datos 3D
+  ####*Parte 3:* Parseo, Inspección y preprocesamiento de datos para PCA
 + ####*Visualización 5:* Intensidad de Pixel
 + ####*Visualización 6:* Datos Normalizados
 + ####*Visualización 7:* Los dos mejores componentes como imágenes
 + ####*Visualización 8:* Los dos mejores componentes como una única imagen
+  ####*Parte 4:* Agregación basada en características seguida de PCA
 + ####*Visualización 9:* Los dos mejores componentes en el tiempo
 + ####*Visualización 10:* Los dos mejores componentes por dirección
 
#### Puedes consultar el API de Spark en [Spark's Python API](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD) y el de NumPy lo tienes en [NumPy Reference](http://docs.scipy.org/doc/numpy/reference/index.html)

 
### **Parte 1: Pasos de un PCA sobre un dataset de muestra**

#### **Visualización 1: Gausianas de dos dimensiones**
#### El análisis de componentes principales o PCA es una estrategia para la reducción de dimensiones. Para entender mejor PCA, trabajaremos con datos sintéticos generados mediante un muestreo de una [distribución bidemensional gausiana](http://en.wikipedia.org/wiki/Multivariate_normal_distribution).  Esta distribución toma como entrada la media y la varianza de cada dimensión y la covarianza entre las dos dimensiones.
 
#### En las visualizaciones de abajo especificamos la media de cada dimensión a 50 y la varianza de cada dimensión la fijamos a 1. Exploraremos dos valores diferentes para la covarianza: 0 y 0.9. Cuando la covarianza es 0, las dos dimensiones no están correladas y por lo tanto los datos parecerán esféricos mientras que si la covarianza es 0.9 ambas dimensiones están fuerte (y positivamente) correladas y por lo tanto los datos no son esféricos. Como veremos en las partes 1 y 2, los datos no esféricos son susceptibles de una reducción dimensional vía PCA.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def preparePlot(xticks, yticks, figsize=(10.5, 6), hideLabels=False, gridColor='#999999',
                gridWidth=1.0):
    """ Plantilla para generar la rejilla."""
    plt.close()
    fig, ax = plt.subplots(figsize=figsize, facecolor='white', edgecolor='white')
    ax.axes.tick_params(labelcolor='#999999', labelsize='10')
    for axis, ticks in [(ax.get_xaxis(), xticks), (ax.get_yaxis(), yticks)]:
        axis.set_ticks_position('none')
        axis.set_ticks(ticks)
        axis.label.set_color('#999999')
        if hideLabels: axis.set_ticklabels([])
    plt.grid(color=gridColor, linewidth=gridWidth, linestyle='-')
    map(lambda position: ax.spines[position].set_visible(False), ['bottom', 'top', 'left', 'right'])
    return fig, ax

def create2DGaussian(mn, sigma, cov, n):
    """ puntos aleatorios de una gausiana bidimensional """
    np.random.seed(142)
    return np.random.multivariate_normal(np.array([mn, mn]), np.array([[sigma, cov], [cov, sigma]]), n)

In [ ]:
dataRandom = create2DGaussian(mn=50, sigma=1, cov=0, n=100)

# generación de la rejila y plot de datos
fig, ax = preparePlot(np.arange(46, 55, 2), np.arange(46, 55, 2))
ax.set_xlabel(r'Simulated $x_1$ values'), ax.set_ylabel(r'Simulated $x_2$ values')
ax.set_xlim(45, 54.5), ax.set_ylim(45, 54.5)
plt.scatter(dataRandom[:,0], dataRandom[:,1], s=14**2, c='#d6ebf2', edgecolors='#8cbfd0', alpha=0.75)
pass

In [ ]:
dataCorrelated = create2DGaussian(mn=50, sigma=1, cov=.9, n=100)

# generación de la rejila y plot de datos
fig, ax = preparePlot(np.arange(46, 55, 2), np.arange(46, 55, 2))
ax.set_xlabel(r'Simulated $x_1$ values'), ax.set_ylabel(r'Simulated $x_2$ values')
ax.set_xlim(45.5, 54.5), ax.set_ylim(45.5, 54.5)
plt.scatter(dataCorrelated[:,0], dataCorrelated[:,1], s=14**2, c='#d6ebf2',
            edgecolors='#8cbfd0', alpha=0.75)
pass

#### **(1a) Interpretación de PCA**
#### Se puede pensar en PCA en la manera de identificar las "direcciones" en las que los datos varían más. En un primer paso del PCA centramos nuestros datos. En nuestro dataset correlado primero calcularemos la media de cada característica (columna). Para cada observación modifica las características restando su media para crear un dataset centrado en la media (media cero) 
#### `correlatedData` es un RDD de arrays de NumPy esto nos permite realizar ciertas operaciones de una manera más sucinta, por ejemplo podemos sumar las columnas de nuestro dataset usando  `correlatedData.sum()`.

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
correlatedData = sc.parallelize(dataCorrelated)

meanCorrelated = <RELLENA>
correlatedDataZeroMean = correlatedData.<RELLENA>

print meanCorrelated
print correlatedData.take(1)
print correlatedDataZeroMean.take(1)

In [ ]:
# TEST Interpretación de PCA (1a)
from test_helper import Test
Test.assertTrue(np.allclose(meanCorrelated, [49.95739037, 49.97180477]),
                'valores incorrectos para meanCorrelated')
Test.assertTrue(np.allclose(correlatedDataZeroMean.take(1)[0], [-0.28561917, 0.10351492]),
                'valores incorrectos para correlatedDataZeroMean')

#### **(1b) Matriz de covarianza de muestra**
#### Ahora estamos listos para calcular la matriz de covarianza de muestra, si definimos  $\scriptsize \mathbf{X} \in \mathbb{R}^{n \times d}$ como la matriz de datos de media cero la matriz de muestra se define como: $$ \mathbf{C}_{\mathbf X} = \frac{1}{n} \mathbf{X}^\top \mathbf{X} \,.$$  Para calcular esta matriz se realiza el producto externo (outer product) para cada punto de datos. Los datos son bidimensionales asi que la matriz de covarianza resultante debería ser una matriz de 2x2.
 
#### Se puede usar [np.outer()](http://docs.scipy.org/doc/numpy/reference/generated/numpy.outer.html) para calcular el outer product de dos arrays NumPy.

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
# Calcula la matriz de covarianza usando productos externos y and correlatedDataZeroMean
correlatedCov = <RELLENA>
print correlatedCov

In [ ]:
# TEST Matriz de covariaza de muestra (1b)
covResult = [[ 0.99558386,  0.90148989], [0.90148989, 1.08607497]]
Test.assertTrue(np.allclose(covResult, correlatedCov), 'valor incorrecto para correlatedCov')

#### **(1c) Función de covarianza**
#### A continuación usaremos la expresión anterior para escribir una función que calcule la matriz de covarianza de muestra para un RDD arbitrario `data`.

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
def estimateCovariance(data):
    """ Calcular la matriz de covarianza para un rdd dado.

    Nota:
        El array de covarianza multi-dimensional debería calcurse usando outer prudcts. No olvides
        Normalizar los datos restando primero la media.

    Args:
        data (RDD of np.ndarray):  Un `RDD` de arrays de NumPy.

    Returns:
        np.ndarray: un array multi-dimensional donde el número de filas y columnas son iguales a la longitud 
        de los arrays en el `RDD` de entrada.
    """
    <RELLENA>

correlatedCovAuto= estimateCovariance(correlatedData)
print correlatedCovAuto

In [ ]:
# TEST Función de Covarianza (1c)
correctCov = [[ 0.99558386,  0.90148989], [0.90148989, 1.08607497]]
Test.assertTrue(np.allclose(correctCov, correlatedCovAuto),
                'valor incorrecto para correlatedCovAuto')

#### **(1d) Eigendescomposición**
#### Ahora que hemos calculado la matriz de covarianza de muestra la podemos usar para encontrar las direcciones de variación máxima de la varianza en los datos, en concreto, podemos realizar una eigendescomposición de esta matriz para encontrar los autovalores y los autovectores.  Los  $\scriptsize d $ autovectores de la matriz de covarianza nos dan las direcciones de máxima varianza y se les suelen llamar "componentes principales". Los autovalores asociados son las varianzas en esas direcciones. En particular el autovector que se corresponde con el mayor autovalor es la dirección de variación máxima (a veces se le llama el autovector principal o "top" egigenvector). La Eigendescomposición de una matriz $\scriptsize d \times d $ de covarianza tiene (grosso modo) una complejidad de ejecución cúbica con respecto a  $\scriptsize d $.  Cuando $\scriptsize d $ es relativamente pequeño (por ejemplo menos que unos pocos miles) podemos realizar la eigendescomposición localmente.

#### Usar la función de `numpy.linalg` llamada [eigh](http://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.eigh.html) para realizar la eigendescomposición. A continuación ordena los autovectores en relación a sus correspondientes autovalores del más alto al más bajo, devolviendo una matriz donde las columnas son los autovectores (y la primera columna es el principal autovector). Puedes usar  [np.argsort](http://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html#numpy-argsort) para obtener los índices de los autovalores que se corresponden con el orden ascendente de los autovalores. Por último asigna la variable `topComponent` al autovector principal (componente principal) que es un vector  $\scriptsize 2 $-dimensional  (array con dos valores).
#### Nota: Los autovectores devueltos por  `eigh` están en columnas, no en filas, Por ejemplo el primer autovector de `eigVecs` se encontraría en la primera columna y se accedería a él usando `eigVecs[:,0]`.

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
from numpy.linalg import eigh

# Calcula los autovalores y autovectores de correlatedCovAuto
eigVals, eigVecs = <RELLENA>
print 'eigenvalues: {0}'.format(eigVals)
print '\neigenvectors: \n{0}'.format(eigVecs)

# Usa np.argsort para encontrar el eigenvector principal basado en el mayor eigenvalue
inds = np.argsort(<RELLENA>)
topComponent = <RELLENA>
print '\nMayor de los componentes principales: {0}'.format(topComponent)

In [ ]:
# TEST Eigendescomposición (1d)
def checkBasis(vectors, correct):
    return np.allclose(vectors, correct) or np.allclose(np.negative(vectors), correct)
Test.assertTrue(checkBasis(topComponent, [0.68915649, 0.72461254]),
                'valor incorrecto para topComponent')

#### **(1e) Puntuaciones PCA**
#### Acabamos de calcular el mayor de  los componentes principales para un dataset no esférico y bidimensional. Ahora vamos a usar este componente principal para obtener una representación unidimensional para los datos originales. Para calcular esta representación compacta que a veces se denomina "puntuaciones PCA" se realiza el dot product entre cada punto de los datos originales y el mayor de los componentes principales.

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
# Usa topComponent y los datos de correlatedData para generar las puntuaciones PCA
correlatedDataScores = <RELLENA>
print 'tres primeros datos unidimensionales:\n{0}'.format(np.asarray(correlatedDataScores.take(3)))

In [ ]:
# TEST puntuaciones PCA (1e)
firstThree = [70.51682806, 69.30622356, 71.13588168]
Test.assertTrue(checkBasis(correlatedDataScores.take(3), firstThree),
                'valor incorreto para correlatedDataScores')

### **Parte 2: Escribir una función PCA y evaluación en el dataset de muestra**

#### **(2a) Función de PCA**
#### Ahora tenemos todos los ingredientes para escribir una función general de PCA. En lugar de trabajar con los componentes principales más altos nuestra función calculará los $\scriptsize k$ componentes principales más altos y las principales puntuaciones para un dataset. Escribe la función general `pca` y ejecútala con `correlatedData` y $\scriptsize k = 2$. Utiliza los resultados de las partes (1c), (1d) y (1e).

#### Recuerda nuestra implementación es una estrategia razonable cuando $\scriptsize d $ es pequeño pero hay algoritmos distribuidos más eficaces cuando $\scriptsize d $ es grande.

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
def pca(data, k=2):
    """ Calcula los `k` componentes principales más altos sus puntuaciones correspondientes
    y todos los autovalores
    
    Note:
        Todos los autovalores deberían devolverse ordenados de mayor a menor. `eigh` devuelve cada
        autovector como una columna. Esta función debería también devolver los autovectores como columnas.

    Args:
        data (RDD de np.ndarray): un `RDD` de arrays NumPy.
        k (int): El número de componentes principales a devolver.

    Returns:
        tuplas de  (np.ndarray, RDD de np.ndarray, np.ndarray): Una tupla de 
        (autovectores, `RDD` de puntuaciones, autovalores). Autovectores es un array multidimensional donde el 
        número de filas es igual a la longitud de los arrays del `RDD` de entrada y el número de columnas es
        igual a `k`. El `RDD` de puntuaciones tiene el mismo número de filas que `data`y consiste de arrays de
        longitud `k`. Los autovalores es un array de longitud d (el número de características).
    """
    <RELLENA>
    # Devuelve las `k`componentes principales, `k` puntuaciones y todos los autovalores
    <RELLENA>

# Ejecuta pca sobre correlatedData con k = 2
topComponentsCorrelated, correlatedDataScoresAuto, eigenvaluesCorrelated = <RELLENA>

# El primer elemento de los componentes principales esta en la primera columna
print 'topComponentsCorrelated: \n{0}'.format(topComponentsCorrelated)
print ('\ncorrelatedDataScoresAuto (first three): \n{0}'
       .format('\n'.join(map(str, correlatedDataScoresAuto.take(3)))))
print '\neigenvaluesCorrelated: \n{0}'.format(eigenvaluesCorrelated)

# Crear un set de test con dimensionalidad más alta
pcaTestData = sc.parallelize([np.arange(x, x + 4) for x in np.arange(0, 20, 4)])
componentsTest, testScores, eigenvaluesTest = pca(pcaTestData, 3)

print '\npcaTestData: \n{0}'.format(np.array(pcaTestData.collect()))
print '\ncomponentsTest: \n{0}'.format(componentsTest)
print ('\ntestScores (first three): \n{0}'
       .format('\n'.join(map(str, testScores.take(3)))))
print '\neigenvaluesTest: \n{0}'.format(eigenvaluesTest)

In [ ]:
# TEST Función de PCA (2a)
Test.assertTrue(checkBasis(topComponentsCorrelated.T,
                           [[0.68915649,  0.72461254], [-0.72461254, 0.68915649]]),
                'valor incorrecto para topComponentsCorrelated')
firstThreeCorrelated = [[70.51682806, 69.30622356, 71.13588168], [1.48305648, 1.5888655, 1.86710679]]
Test.assertTrue(np.allclose(firstThreeCorrelated,
                            np.vstack(np.abs(correlatedDataScoresAuto.take(3))).T),
                'valor incorrecto para firstThreeCorrelated')
Test.assertTrue(np.allclose(eigenvaluesCorrelated, [1.94345403, 0.13820481]),
                           'valores incorrectos para eigenvaluesCorrelated')
topComponentsCorrelatedK1, correlatedDataScoresK1, eigenvaluesCorrelatedK1 = pca(correlatedData, 1)
Test.assertTrue(checkBasis(topComponentsCorrelatedK1.T, [0.68915649,  0.72461254]),
               'valores incorrectos para los componentes cuando k=1')
Test.assertTrue(np.allclose([70.51682806, 69.30622356, 71.13588168],
                            np.vstack(np.abs(correlatedDataScoresK1.take(3))).T),
                'valor incorrecto para las puntuaciones cuando k=1')
Test.assertTrue(np.allclose(eigenvaluesCorrelatedK1, [1.94345403, 0.13820481]),
                           'valores incorrectos para los autovalores cuando k=1')
Test.assertTrue(checkBasis(componentsTest.T[0], [ .5, .5, .5, .5]),
                'valor incorrecto para componentsTest')
Test.assertTrue(np.allclose(np.abs(testScores.first()[0]), 3.),
                'valor incorrecto para testScores')
Test.assertTrue(np.allclose(eigenvaluesTest, [ 128, 0, 0, 0 ]), 'valor incorrecto para eigenvaluesTest')

#### **(2b) PCA en `dataRandom`**
#### A continuación usa la función PCA para encontrar los dos componentes principales más altos del `dataRandom` esférico que creamos en la visualización 1.

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
randomData = sc.parallelize(dataRandom)

# Usa pca en randomData
topComponentsRandom, randomDataScoresAuto, eigenvaluesRandom = <RELLENA>

print 'topComponentsRandom: \n{0}'.format(topComponentsRandom)
print ('\nrandomDataScoresAuto (tres primeros): \n{0}'
       .format('\n'.join(map(str, randomDataScoresAuto.take(3)))))
print '\neigenvaluesRandom: \n{0}'.format(eigenvaluesRandom)

In [ ]:
# TEST PCA en `dataRandom` (2b)
Test.assertTrue(checkBasis(topComponentsRandom.T,
                           [[-0.2522559 ,  0.96766056], [-0.96766056,  -0.2522559]]),
                'valor incorrecto para topComponentsRandom')
firstThreeRandom = [[36.61068572,  35.97314295,  35.59836628],
                    [61.3489929 ,  62.08813671,  60.61390415]]
Test.assertTrue(np.allclose(firstThreeRandom, np.vstack(np.abs(randomDataScoresAuto.take(3))).T),
                'valor incorrecto par randomDataScoresAuto')
Test.assertTrue(np.allclose(eigenvaluesRandom, [1.4204546, 0.99521397]),
                            'valor incorrecto para eigenvaluesRandom')

#### **Visualización 2: Proyección PCA**
#### Traza los datos originales y la reconstrucción unidimensional usando el componente principal más alto para ver como es la solución con PCA. Los datos originales se pintan como antes y la reconstrucción unidimensional (proyección) se pinta en verde sobre los datos originales y los vectores (líneas) que representan los dos componentes principales que se muestran como líneas punteadas.

In [ ]:
def projectPointsAndGetLines(data, components, xRange):
    """ Proyecta los datos originales en la primera componente y obtiene detalles de la línea par los dos
    componentes más altos."""
    topComponent= components[:, 0]
    slope1, slope2 = components[1, :2] / components[0, :2]

    means = data.mean()[:2]
    demeaned = data.map(lambda v: v - means)
    projected = demeaned.map(lambda v: (v.dot(topComponent) /
                                        topComponent.dot(topComponent)) * topComponent)
    remeaned = projected.map(lambda v: v + means)
    x1,x2 = zip(*remeaned.collect())

    lineStartP1X1, lineStartP1X2 = means - np.asarray([xRange, xRange * slope1])
    lineEndP1X1, lineEndP1X2 = means + np.asarray([xRange, xRange * slope1])
    lineStartP2X1, lineStartP2X2 = means - np.asarray([xRange, xRange * slope2])
    lineEndP2X1, lineEndP2X2 = means + np.asarray([xRange, xRange * slope2])

    return ((x1, x2), ([lineStartP1X1, lineEndP1X1], [lineStartP1X2, lineEndP1X2]),
            ([lineStartP2X1, lineEndP2X1], [lineStartP2X2, lineEndP2X2]))

In [ ]:
((x1, x2), (line1X1, line1X2), (line2X1, line2X2)) = \
    projectPointsAndGetLines(correlatedData, topComponentsCorrelated, 5)

# genera el layout y dibuja los datos
fig, ax = preparePlot(np.arange(46, 55, 2), np.arange(46, 55, 2), figsize=(7, 7))
ax.set_xlabel(r'Simulated $x_1$ values'), ax.set_ylabel(r'Simulated $x_2$ values')
ax.set_xlim(45.5, 54.5), ax.set_ylim(45.5, 54.5)
plt.plot(line1X1, line1X2, linewidth=3.0, c='#8cbfd0', linestyle='--')
plt.plot(line2X1, line2X2, linewidth=3.0, c='#d6ebf2', linestyle='--')
plt.scatter(dataCorrelated[:,0], dataCorrelated[:,1], s=14**2, c='#d6ebf2',
            edgecolors='#8cbfd0', alpha=0.75)
plt.scatter(x1, x2, s=14**2, c='#62c162', alpha=.75)
pass

In [ ]:
((x1, x2), (line1X1, line1X2), (line2X1, line2X2)) = \
    projectPointsAndGetLines(randomData, topComponentsRandom, 5)

# genera el layout y dibuja los datos
fig, ax = preparePlot(np.arange(46, 55, 2), np.arange(46, 55, 2), figsize=(7, 7))
ax.set_xlabel(r'Simulated $x_1$ values'), ax.set_ylabel(r'Simulated $x_2$ values')
ax.set_xlim(45.5, 54.5), ax.set_ylim(45.5, 54.5)
plt.plot(line1X1, line1X2, linewidth=3.0, c='#8cbfd0', linestyle='--')
plt.plot(line2X1, line2X2, linewidth=3.0, c='#d6ebf2', linestyle='--')
plt.scatter(dataRandom[:,0], dataRandom[:,1], s=14**2, c='#d6ebf2',
            edgecolors='#8cbfd0', alpha=0.75)
plt.scatter(x1, x2, s=14**2, c='#62c162', alpha=.75)
pass

#### **Visualización 3: Datos tridimensionales**
#### Hasta ahora hemos trabajado con datos bidimensionales, vamos a generar ahora datos tridimensionales con características altamente correladas. Como en la visualización 1, crearemos muestras de una distribución gausiana multivariante que requiere especificar tres medias, tres varianzas y tres covarianzas.

#### En los gráficos en 3D que se muestran abajo hemos incluido el plano en 2D que se corresponde con las dos componentes principales más altas (el plano con la menor distancia euclídea entre los puntos y el mismo). Los puntos de datos aunque están en un espacio tridimensional se encuentran cerca del plano 2D: El gráfico de la izquierda muestra como la mayoria de los puntos estan cerca del plano, el gráfico de la derecha muestra como el plano cubre la mayor parte de la varianza de los datos. Los puntos azul oscuro son los valores más altos en la tercera dimensión.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

m = 100
mu = np.array([50, 50, 50])
r1_2 = 0.9
r1_3 = 0.7
r2_3 = 0.1
sigma1 = 5
sigma2 = 20
sigma3 = 20
c = np.array([[sigma1 ** 2, r1_2 * sigma1 * sigma2, r1_3 * sigma1 * sigma3],
             [r1_2 * sigma1 * sigma2, sigma2 ** 2, r2_3 * sigma2 * sigma3],
             [r1_3 * sigma1 * sigma3, r2_3 * sigma2 * sigma3, sigma3 ** 2]])
np.random.seed(142)
dataThreeD = np.random.multivariate_normal(mu, c, m)

from matplotlib.colors import ListedColormap, Normalize
from matplotlib.cm import get_cmap
norm = Normalize()
cmap = get_cmap("Blues")
clrs = cmap(np.array(norm(dataThreeD[:,2])))[:,0:3]

fig = plt.figure(figsize=(11, 6))
ax = fig.add_subplot(121, projection='3d')
ax.azim=-100
ax.scatter(dataThreeD[:,0], dataThreeD[:,1], dataThreeD[:,2], c=clrs, s=14**2)

xx, yy = np.meshgrid(np.arange(-15, 10, 1), np.arange(-50, 30, 1))
normal = np.array([0.96981815, -0.188338, -0.15485978])
z = (-normal[0] * xx - normal[1] * yy) * 1. / normal[2]
xx = xx + 50
yy = yy + 50
z = z + 50

ax.set_zlim((-20, 120)), ax.set_ylim((-20, 100)), ax.set_xlim((30, 75))
ax.plot_surface(xx, yy, z, alpha=.10)

ax = fig.add_subplot(122, projection='3d')
ax.azim=10
ax.elev=20
#ax.dist=8
ax.scatter(dataThreeD[:,0], dataThreeD[:,1], dataThreeD[:,2], c=clrs, s=14**2)

ax.set_zlim((-20, 120)), ax.set_ylim((-20, 100)), ax.set_xlim((30, 75))
ax.plot_surface(xx, yy, z, alpha=.1)
plt.tight_layout()
pass

#### **(2c) De 3D a 2D**
#### Usaremos ahora PCA para ver si podemos recuperar el plano 2D en donde están los datos. Paraleliza los datos y utiliza la función PCA previa con  $ \scriptsize k=2 $ componentes.

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
threeDData = sc.parallelize(dataThreeD)
componentsThreeD, threeDScores, eigenvaluesThreeD = <RELLENA>

print 'componentsThreeD: \n{0}'.format(componentsThreeD)
print ('\nthreeDScores (first three): \n{0}'
       .format('\n'.join(map(str, threeDScores.take(3)))))
print '\neigenvaluesThreeD: \n{0}'.format(eigenvaluesThreeD)

In [ ]:
# TEST 3D to 2D (2c)
Test.assertEquals(componentsThreeD.shape, (3, 2), 'forma incorrecta de componentsThreeD')
Test.assertTrue(np.allclose(np.sum(eigenvaluesThreeD), 969.796443367),
                'valor incorrecto para eigenvaluesThreeD')
Test.assertTrue(np.allclose(np.abs(np.sum(componentsThreeD)), 1.77238943258),
                'valor incorrecto para componentsThreeD')
Test.assertTrue(np.allclose(np.abs(np.sum(threeDScores.take(3))), 237.782834092),
                'valor incorrecto para threeDScores')

#### **Visualización 4: respresentación en 2D de datos 3D**
#### Veamos como la versión en 2D de los datos captura la mayor parte de la estructura original. Los puntos azul oscuro se corresponden con los valores más altos de los datos en la 3ª dimensión.

In [ ]:
scoresThreeD = np.asarray(threeDScores.collect())

# genera la plantilla y pinta los datos
fig, ax = preparePlot(np.arange(20, 150, 20), np.arange(-40, 110, 20))
ax.set_xlabel(r'New $x_1$ values'), ax.set_ylabel(r'New $x_2$ values')
ax.set_xlim(5, 150), ax.set_ylim(-45, 50)
plt.scatter(scoresThreeD[:,0], scoresThreeD[:,1], s=14**2, c=clrs, edgecolors='#8cbfd0', alpha=0.75)
pass

#### **(2d) Explicación de la varianza**
#### Por último vamos a cuantificar cuanta varianza se está capturando por PCA en cada unos de los 3 datasets sintéticos que hemos analizado. Para hacerlo calcularemos la fracción de la varianza retenida en los componentes principales más altos. Recuerda que los autovalores correspondientes a cada componente principal captura la varianza a lo largo de esa dirección. Si nuestros datos iniciales son  $\scriptsize d$-dimensionales, la varianza total en nuestros datos es igual a: $ \scriptsize \sum_{i=1}^d \lambda_i $, donde $\scriptsize \lambda_i$ es el autovalor correspondiente al  $\scriptsize i$-ésimo componente principal. Es más, si usamos PCA con algún  $\scriptsize k < d$ entonces podemos calcular la varianza retenida por estos componentes principales añadiendo los $\scriptsize k$ autovalores más altos. La fracción de la varianza retenida es igual a la suma de los  $\scriptsize k$ autovalores más altos dividida por la suma de todos los autovalores.

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
def varianceExplained(data, k=1):
    """Calcula la fracción de la varianza explicada por los `k` autovectores más altos.

    Args:
        data (RDD of np.ndarray): un RDD que contiene arrays NumPy que almacenan las características de una 
            observación.
        k: El número de componentes principales a considerar.

    Returns:
        float: Un número entro 0 y 1 que representa el porcentaje de la varianza explicada por los `k` autovalores
            más altos.
    """
    components, scores, eigenvalues = <RELLENA>
    <RELLENA>

varianceRandom1 = varianceExplained(randomData, 1)
varianceCorrelated1 = varianceExplained(correlatedData, 1)
varianceRandom2 = varianceExplained(randomData, 2)
varianceCorrelated2 = varianceExplained(correlatedData, 2)
varianceThreeD2 = varianceExplained(threeDData, 2)
print ('Porcentaje de la varianza explicada por el primer componente de randomData: {0:.1f}%'
       .format(varianceRandom1 * 100))
print ('Porcentaje de la varianza explicada por ambos componentes de randomData: {0:.1f}%'
       .format(varianceRandom2 * 100))
print ('\nPorcentaje de la varianza explicada por el primer componentes de correlatedData: {0:.1f}%'.
       format(varianceCorrelated1 * 100))
print ('Porcentaje de la varianza explicada por ambos componentes de correlatedData: {0:.1f}%'
       .format(varianceCorrelated2 * 100))
print ('\nPorcentaje de la varianza explicada por el primero de los los dos componentes de threeDData: {0:.1f}%'
       .format(varianceThreeD2 * 100))

In [ ]:
# TEST Explicación de la varianza (2d)
Test.assertTrue(np.allclose(varianceRandom1, 0.588017172066), 'valor incorrecto para varianceRandom1')
Test.assertTrue(np.allclose(varianceCorrelated1, 0.933608329586),
                'valor incorrecto para varianceCorrelated1')
Test.assertTrue(np.allclose(varianceRandom2, 1.0), 'valor incorrecto para varianceRandom2')
Test.assertTrue(np.allclose(varianceCorrelated2, 1.0), 'valor incorrecto para varianceCorrelated2')
Test.assertTrue(np.allclose(varianceThreeD2, 0.993967356912), 'valor incorrecto para varianceThreeD2')

 
### **Parte 3: Parseo, inspección y preprocesamiento de datos para PCA**

 
#### **Introducción a los datos**
#### Un reto importante para la neurociencia es el entender como están organizadas y como funcionan las neuronas que son las células responsables del procesamiento y representación de la información en el cerebro. Las nuevas tecnología hacen posible monitorizar la respuesta de grandes cantidades de neuronas en animales vivos. Las neuronas se comunican mediante impulsos eléctricos que son registrados con electrodos lo cual no siempre es fácil de hacer, como alternativa se puede modificar geneticamente animales para que dispongan de proteinas especiales luminiscentes que se activan cuando hay actividad en las neuronas y utilizar un microscopio para registrar esa actividad neuronal. Recientemente se ha desarrollado un método llamado microscopía light-sheet que permite en un animal transparente como el pez cebra registrar todo su cerebro. Los datos resultantes son imágenes a lo largo de un tiempo de medida que contienen la actividad de miles de neuronas. El dataset en bruto es enorme y la idea es encontrar patrones de representación más compactos tanto espaciales como temporales: ¿Qué grupos de neuronas se activan juntas? ¿Cuál es la evolución temporal de su actividad? ¿Son esos patrones específicos a los eventos particulares que suceden durante el experimento ? PCA es una herramienta potente para encontrar patrones espaciales y temporales en esta clase de datos asi que ¡La vamos utilizar!

#### **(3a) Carga de datos**
#### En las siguientes secciones usaremos PCA para capturar la estructura de datasets neuronales. Antes de hacer el análisis cargaremos y haremos alguna inspección básica sobre los datos. Los datos en bruto están almacenados en un fichero de texto. Cada línea in el fichero contiene las series temporales de cambios de intensidad de un único pixel sobre una imagen que van variando con el tiempo (como en una película). Los dos primeros números de cada línea son las coordenadas espaciales del pixel y el resto de valores son la serie temporal. usa first() para inpeccionar una única fila e imprime rlos 100 primeros caracteres.

In [ ]:
import os
baseDir = os.path.join('data')
inputPath = os.path.join('utad-spark-ml', 'neuro.txt')

inputFile = os.path.join(baseDir, inputPath)

lines = sc.textFile(inputFile)
print lines.first()[0:100]

# comprobación de carga correcta
assert len(lines.first()) == 1397
assert lines.count() == 46460

#### **(3b) Parseo de los datos**
#### Vamos a parsear los datos en una representación clave-valor. Queremos que cada clave sea una tupla de las coordenadas espaciales bidimensionales y cada valor sea un array NumPy que almacene la serie temporal asociada. Escribe una función que convierta una línea de texto en un par (`tuple`, `np.ndarray`) y aplica esta función a cada registro en el RDD e inspecciona la primera entrada en el nuevo dataset parseado. Ahora sería un buen momento para cachear los datos y forzar el cálculo llamando a count para asegurarnos de que los datos se han cacheado.

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
def parse(line):
    """ Parsea los datos en bruto en un par (`tuple`, `np.ndarray`).

    Note:
        Deberías almacenar las coordenadas del pixel en una tupla de dos interos y los elementos  de la serie
        temporal de la intensidad del pixe. como floats en un np.ndarray.

    Args:
        line (str): Un string que representa una observación. Los elementos estan separados por espacios.
            Los primeros dos elementos representan las coordenadas del pixel y el resto de los elementos 
            representan la intensidad del pixel a lo largo del tiempo.

    Returns:
        tuple of tuple, np.ndarray: una `tupla` (coordenada, array de intensidad del pixel) donde coordenada es
        una `tupla` que contiene dos valores y la intensidad del pixel se almacena en un array NumPy que contiene
        240 valores.
    """
    <RELLENA>

rawData = lines.map(parse)
rawData.cache()
entry = rawData.first()
print 'La longitud de la película es de {0} segundos'.format(len(entry[1]))
print 'El número de pixels en la pelicula es de {0:,}'.format(rawData.count())
print ('\nPrimera entrada de los datos en bruto (sólo los 5 primeros valores):\n({0}, {1})'
       .format(entry[0], entry[1][:5]))

In [ ]:
# TEST Parseo de datos (3b)
Test.assertTrue(isinstance(entry[0], tuple), "La clave debería ser una tupla")
Test.assertEquals(len(entry), 2, 'La entrada deberia tener una entrada y un valor')
Test.assertTrue(isinstance(entry[0][1], int), 'la tupla de coordenadas deberia contener enteros')
Test.assertEquals(len(entry[0]), 2, "La clave debería tener dos valores")
Test.assertTrue(isinstance(entry[1], np.ndarray), "los valores debería ser un np.ndarray")
Test.assertTrue(isinstance(entry[1][0], np.float), 'el np.ndarray debe estar compuesto por floats')
Test.assertEquals(entry[0], (0, 0), 'clave incorrecta ')
Test.assertEquals(entry[1].size, 240, 'longitud de array incorrecta')
Test.assertTrue(np.allclose(np.sum(entry[1]), 24683.5), 'valores incorrectos en el array')

#### **(3c) Fluorescencia Mínima y Máxima**
#### A continuación vamos a hacer un preprocesado básico de los datos. Los datos de la serie temporal en bruto están  en unidades de fluorenscencia y la baseline de fluerescencia varía de manera arbitraria de un pixel a otro. Primeros calcula el mínimo y el máximo  de fluorescencia de todos los pixels.

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
mn = <RELLENA>
mx = <RELLENA>

print mn, mx

In [ ]:
# TEST Min y max de fluorescencia(3c)
Test.assertTrue(np.allclose(mn, 100.6), 'valor incorrecto para mn')
Test.assertTrue(np.allclose(mx, 940.8), 'valor incorrecto para mx')

#### **Visualización 5: Intensidad de Pixel**
#### Vamos a ver como un pixel aleatorio cambia de valor a lo largo de la serie temporal. Tomaremos un pixel que tenga una desviación normal de mas de 100.

In [ ]:
example = rawData.filter(lambda (k, v): np.std(v) > 100).values().first()

# generar la regilla y dibujar los datos
fig, ax = preparePlot(np.arange(0, 300, 50), np.arange(300, 800, 100))
ax.set_xlabel(r'time'), ax.set_ylabel(r'flouresence')
ax.set_xlim(-20, 270), ax.set_ylim(270, 730)
plt.plot(range(len(example)), example, c='#8cbfd0', linewidth='3.0')
pass

#### **(3d) Cambio de señal fraccionada**
#### Para convertir las unidades de fluorescencia en unidades de cambio de señal fraccionada, escribe una función que tome una serie temporal para un pixel en particular y le reste y le divida la media, después aplica esta función a todos los pixels. Confirma que el los valores máximo y mínimo han cambiado.

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
def rescale(ts):
    """ Toma un np.ndarray y devuelve el array normalizado restando y dividiendo por la media.n.

    Note:
        Primero resta la media y después divídelo por la media.

    Args:
        ts (np.ndarray): Serie temporal de (`np.float`) que representa la intensidad del pixel.

    Returns:
        np.ndarray: La serie temporal ajustada restando la media y dividiendo por la media.
    """
    <RELLENA>

scaledData = rawData.mapValues(lambda v: rescale(v))
mnScaled = scaledData.map(lambda (k, v): v).map(lambda v: min(v)).min()
mxScaled = scaledData.map(lambda (k, v): v).map(lambda v: max(v)).max()
print mnScaled, mxScaled

In [ ]:
# TEST Cambio de señal fraccionada (3d)
Test.assertTrue(isinstance(scaledData.first()[1], np.ndarray), 'tipo incorrecto en la rescala')
Test.assertTrue(np.allclose(mnScaled, -0.27151288), 'valor incorrecto para mnScaled')
Test.assertTrue(np.allclose(mxScaled, 0.90544876), 'valor incorrecto para mxScaled')

#### **Visualización 6: Datos Normalizados**
#### Ahora que hemos normalizado los datos veamos otra vez como el pixel aleatorio cambia de valor a lo largo de la serie temporal.  Vamos a visualizar un pixel que tenga una desviación estandar de más de 0.1. Nota el cambio de escala en el eje de las y comparado con la anterior visualización.

In [ ]:
example = scaledData.filter(lambda (k, v): np.std(v) > 0.1).values().first()

# Genera la parrilla y dibuja los datos
fig, ax = preparePlot(np.arange(0, 300, 50), np.arange(-.1, .6, .1))
ax.set_xlabel(r'time'), ax.set_ylabel(r'flouresence')
ax.set_xlim(-20, 260), ax.set_ylim(-.12, .52)
plt.plot(range(len(example)), example, c='#8cbfd0', linewidth='3.0')
pass

#### **(3e) PCA en los datos escalados**
#### Ahora que hemos preprocesado  el dataset con $\scriptsize n = 46460$ pixels y $\scriptsize d = 240$ segundos en una serie temporal para cada pixel podemos interpretar los pixels como nuestras observaciones y cada valor de la serie temporal como una característica. Nos gustaría encontrar patrones en la actividad cerebral durante esta serie temporal y encontrar correlaciones a lo largo del tiempo. Podemos utilizar PCA para encontrar una representación más compacta de nuestros datos que nos permitan visualizarlos.

#### Usa la función `pca` del apartado (2a) para realizar PCA  en el dataset de neurociencia con  $\scriptsize k = 3$. La función `pca` toma un RDD de arrays pero `data` es un RDD de pares clave-valor asi que necesitarás extraer los valores.

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
# Ejecuta pca usando scaledData
componentsScaled, scaledScores, eigenvaluesScaled = <RELLENA>

In [ ]:
# TEST PCA en los datos escalados (3e)
Test.assertEquals(componentsScaled.shape, (240, 3), 'forma incorrecta para componentsScaled')
Test.assertTrue(np.allclose(np.abs(np.sum(componentsScaled[:5, :])), 0.283150995232),
                'valor incorrecto para componentsScaled')
Test.assertTrue(np.allclose(np.abs(np.sum(scaledScores.take(3))), 0.0285507449251),
                'valor incorrecto para scaledScores')
Test.assertTrue(np.allclose(np.sum(eigenvaluesScaled[:5]), 0.206987501564),
                'valor incorrecto para eigenvaluesScaled')

#### **Visualización 7: Los dos mejores componentes como imágenes**
#### Ahora veremos las puntuaciones para los dos componentes más altos como imágenes. Transformamos los vectores de sus dimensiones originales a un tamaño de imagen de 230 x 202

#### Estos gráficos mapean los valores para un único componente a una imagen en escala de grises lo que nos proporciona una representación visual para poder ver la estructura general del cerebro del pez cebra e identificar donde ocurren los valores más altos y más bajos. No obstante hay una cantidad substancial de información útil que es dificil de interpretar. En la siguiente visualización veremos como podemos mejorar la interpretación combinando las dos componentes principales más altas en una única imagen utilizando un mapa de colores.

In [ ]:
import matplotlib.cm as cm

scoresScaled = np.vstack(scaledScores.collect())
imageOneScaled = scoresScaled[:,0].reshape(230, 202).T

# generar la parrillar y dibujar los datos
fig, ax = preparePlot(np.arange(0, 10, 1), np.arange(0, 10, 1), figsize=(9.0, 7.2), hideLabels=True)
ax.grid(False)
ax.set_title('Top Principal Component', color='#888888')
image = plt.imshow(imageOneScaled,interpolation='nearest', aspect='auto', cmap=cm.gray)
pass

In [ ]:
imageTwoScaled = scoresScaled[:,1].reshape(230, 202).T

# generar la parrillar y dibujar los datos
fig, ax = preparePlot(np.arange(0, 10, 1), np.arange(0, 10, 1), figsize=(9.0, 7.2), hideLabels=True)
ax.grid(False)
ax.set_title('Second Principal Component', color='#888888')
image = plt.imshow(imageTwoScaled,interpolation='nearest', aspect='auto', cmap=cm.gray)
pass

#### **Visualización 8: Los dos mejores componentes como una única imagen**
#### Cuando realizamos PCA y damos color a las neuronas basadas en su localización en un espacio de pocas dimesiones (2/3 dimensiones) podemos interpretar áreas con colores similares como zonas que muestran respuestas similares (al menos en términos de una representación simple que podamos recuperar con PCA). Abajo se muestra en el primer gráfico las dos componentes principales mapeadas a colores. Y en el segundo y tercer gráficos se muestran el resultado de este mapeo de color sobre todos los datos neuronales del pez cebra.
 
#### El segundo y tercer gráficos también se muestran claramente los patrones de similitud neuronal a lo largo de diferentes regiones del cerebro. Sin embargo cuando se realiza PCA sobre todo el dataset hay múltiples razones por las que las neuronas pueden tener respuestas similares. Las neuronas pueden responder de manera similar a diferentes estímulos, sus respuestas pueden tener dinámicas temporales similares o su similitud en la respuesta podría estar influenciada tanto por factores específicos de los estímulos como temporales. No obstante con nuestro análisis inicial PCA no podemos precisar los factores subyacentes y por lo tanto es dificil interpretar que significa realmente "similitud".

#### Detalles opcionales: Usamos [coordenadas polares](https://en.wikipedia.org/wiki/Polar_coordinate_system) para mapear los puntos de colores. Para usar coordenadas polares es necesario proporcionar un ángulo $ (\phi) $ y una magnitud $ (\rho) $. Después vamos a usar el espacio de color polar HSV [hue-saturation-value](https://en.wikipedia.org/wiki/HSL_and_HSV) para mapear el ángulo a una tonalidad (hue)  y la magnitud a un valor (brillo).

In [ ]:
# Adapted from python-thunder's Colorize.transform where cmap='polar'.
# Checkout the library at: https://github.com/thunder-project/thunder and
# http://thunder-project.org/

def polarTransform(scale, img):
    """Convert points from cartesian to polar coordinates and map to colors."""
    from matplotlib.colors import hsv_to_rgb

    img = np.asarray(img)
    dims = img.shape

    phi = ((np.arctan2(-img[0], -img[1]) + np.pi/2) % (np.pi*2)) / (2 * np.pi)
    rho = np.sqrt(img[0]**2 + img[1]**2)
    saturation = np.ones((dims[1], dims[2]))

    out = hsv_to_rgb(np.dstack((phi, saturation, scale * rho)))

    return np.clip(out * scale, 0, 1)

In [ ]:
# Muestra la transformación polar de los componentes principales a colores.
x1AbsMax = np.max(np.abs(imageOneScaled))
x2AbsMax = np.max(np.abs(imageTwoScaled))

numOfPixels = 300
x1Vals = np.arange(-x1AbsMax, x1AbsMax, (2 * x1AbsMax) / numOfPixels)
x2Vals = np.arange(x2AbsMax, -x2AbsMax, -(2 * x2AbsMax) / numOfPixels)
x2Vals.shape = (numOfPixels, 1)

x1Data = np.tile(x1Vals, (numOfPixels, 1))
x2Data = np.tile(x2Vals, (1, numOfPixels))

# Try changing the first parameter to lower values
polarMap = polarTransform(2.0, [x1Data, x2Data])

gridRange = np.arange(0, numOfPixels + 25, 25)
fig, ax = preparePlot(gridRange, gridRange, figsize=(9.0, 7.2), hideLabels=True)
image = plt.imshow(polarMap, interpolation='nearest', aspect='auto')
ax.set_xlabel('Principal component one'), ax.set_ylabel('Principal component two')
gridMarks = (2 * gridRange / float(numOfPixels) - 1.0)
x1Marks = x1AbsMax * gridMarks
x2Marks = -x2AbsMax * gridMarks
ax.get_xaxis().set_ticklabels(map(lambda x: '{0:.1f}'.format(x), x1Marks))
ax.get_yaxis().set_ticklabels(map(lambda x: '{0:.1f}'.format(x), x2Marks))
pass

In [ ]:
# Misma transformación sobre los datos de la imagen con el primer parámetros establecido a valores más bajos
brainmap = polarTransform(2.0, [imageOneScaled, imageTwoScaled])

# generación de parrilla y dibujo de datos
fig, ax = preparePlot(np.arange(0, 10, 1), np.arange(0, 10, 1), figsize=(9.0, 7.2), hideLabels=True)
ax.grid(False)
image = plt.imshow(brainmap,interpolation='nearest', aspect='auto')
pass

### **Parte 4: Agregación basada en características seguida de PCA**

#### **(4a) Agregación usando arrays**
 
#### En el análisis de la parte 3 realizamos PCA sobre toda la serie temporal intentando encontrar patrones globales durante los 240 segundos. Sin embargo nuestro análisis no tiene en cuenta el hecho de que han sucedido diferentes eventos durante esos 240 segundos. En concreto  se le han presentado al pez cebra 12 patrones visuales y cada uno de ellos ha durado 20 segundos haciendo un total de 12 x 20 = 240 características. Seguramente tendremos tendremos mejores patrones si incorporamos el conocimiento de los experimentos dentro de los análisis. Como veremos podemos aislar el impacto de la respuesta temporal agregando de manera apropiada las características.
 
#### Para agregar las características vamos a usar multiplicación de matrices, si usamos  `np.dot` en un array bidimensional NumPy el resultado sería equivalente a una multiplicación de matrices. Por ejemplo `np.array([[1, 2, 3], [4, 5, 6]]).dot(np.array([2, 0, 1]))` devuelve `np.array([5, 14])`.
#### $$\begin{bmatrix} 1 & 2 & 3 \\\ 4 & 5 & 6 \end{bmatrix} \begin{bmatrix} 2 \\\ 0 \\\ 1 \end{bmatrix} = \begin{bmatrix} 5 \\\ 14 \end{bmatrix} $$
#### Si se crea correctamente el array multidimensional podemos multiplicarlo por un vector para realizar operaciones de agregación. Imagina, por ejemplo, que tenemos un vector de 3 dimensiones $ \scriptsize \begin{bmatrix} 1 & 2 & 3 \end{bmatrix}^\top $  y queremos crear un vector de dos dimensiones que contenga la suma de su primer y último elementos como un único valor y tres veces su segundo valor como otro valor i.e. $ \scriptsize \begin{bmatrix} 4 & 6 \end{bmatrix}^\top $. Podemos obtener este resultado haciendo una multiplicación de matrices:  `np.array([[1, 0, 1], [0, 3, 0]]).dot(np.array([1, 2, 3])` lo que devuelve `np.array([4, 6]`.
#### $$\begin{bmatrix} 1 & 0 & 1 \\\ 0 & 3 & 0 \end{bmatrix} \begin{bmatrix} 1 \\\ 2 \\\ 3 \end{bmatrix} = \begin{bmatrix} 4 \\\ 6 \end{bmatrix} $$
#### En este ejercicio crearás varios arrays que realicen diferentes tipos de agregación. La agregación se especifica en los comentarios de cada array. Rellena primero los valores del array a mano y automatizaremos la creación del array en los siguientes dos ejercicios.

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
vector = np.array([0., 1., 2., 3., 4., 5.])

#Crea un array multidimensional que al multiplicarlo por un vector (usando .dot) devuelva un
#array de dos elementos donde el primer elemento sea la suma de los elementos de los índices 0, 2 y 4
#del vector y el segundo elemento sea la suma de los elmentos de los índices 1, 3, y 5.
#Debería ser un array de 2 filas y 6 columnas
sumEveryOther = np.array(<RELLENA>)

#crea un array multidimensional que al multiplicarlo por un vector (usando .dot) devuelva un array de 3 elementos
#donde el primer elemento sea la suma de los elementos de los índices 0 y 3, el segundo elemento sea la suma
#de los elementos de los índices 1 y 4 y el tercer elemento sea la suma de los elementos de los índices 2 y 5
#debería de ser un array de 3 filas y 6 columnas
sumEveryThird = np.array(<RELLENA>)

#Crea un array multidimensional que sirva para sumar los primeros 3 elementos y los
#y 3 últimos elementos del vector y que devuelva un array de dos elementos utilizando .dot
#debería ser un array de 2 filas y 6 columnas

sumByThree = np.array(<RELLENA>)

#Crea un array multidimensional que sume los primeros dos elementos, los segundos dos elementos y
#los últimos dos elementos y que devuelve un array de 3 elementos utilizando .dot
#debería ser un array de 3 filas y 6 columnas

sumByTwo = np.array(<RELLENA>)

print 'sumEveryOther.dot(vector):\t{0}'.format(sumEveryOther.dot(vector))
print 'sumEveryThird.dot(vector):\t{0}'.format(sumEveryThird.dot(vector))

print '\nsumByThree.dot(vector):\t{0}'.format(sumByThree.dot(vector))
print 'sumByTwo.dot(vector): \t{0}'.format(sumByTwo.dot(vector))

In [ ]:
# TEST Agregación usando arrays (4a)
Test.assertEquals(sumEveryOther.shape, (2, 6), 'forma incorrecta para sumEveryOther')
Test.assertEquals(sumEveryThird.shape, (3, 6), 'forma incorrecta para sumEveryThird')
Test.assertTrue(np.allclose(sumEveryOther.dot(vector), [6, 9]), 'valor incorrecto para sumEveryOther')
Test.assertTrue(np.allclose(sumEveryThird.dot(vector), [3, 5, 7]),
                'valor incorrecto para sumEveryThird')
Test.assertEquals(sumByThree.shape, (2, 6), 'valor incorrecto para sumByThree')
Test.assertEquals(sumByTwo.shape, (3, 6), 'valor incorrecto para sumByTwo')
Test.assertTrue(np.allclose(sumByThree.dot(vector),  [3, 12]), 'valor incorrecto para sumByThree')
Test.assertTrue(np.allclose(sumByTwo.dot(vector), [1, 5, 9]), 'valor incorrecto para sumByTwo')

#### **(4b) Uso de `np.tile` y `np.eye`**
#### [np.tile](http://docs.scipy.org/doc/numpy/reference/generated/numpy.tile.html) es útil para repetir array en una o más dimensiones, por ejemplo, `np.tile(np.array([[1, 2], [3, 4]]), 2)` devuelve `np.array([[1, 2, 1, 2], [3, 4, 3, 4]]))`.
####  $$ np.tile( \begin{bmatrix} 1 & 2 \\\ 3 & 4 \end{bmatrix} , 2) \to \begin{bmatrix} 1 & 2 & 1& 2 \\\ 3 & 4 & 3 & 4 \end{bmatrix} $$
#### [np.eye](http://docs.scipy.org/doc/numpy/reference/generated/numpy.eye.html) se puede usar para crear el array identidad $ (\mathbf{I_n}) $.  Por ejemplo, `np.eye(3)` devuelve `np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])`.
#### $$ np.eye( 3 ) \to \begin{bmatrix} 1 & 0 & 0 \\\ 0 & 1 & 0 \\\ 0 & 0 & 1 \end{bmatrix} $$
#### En este ejercicio vamos a implementar  `sumEveryOther` y `sumEveryThird` usando `np.tile` y `np.eye`.

In [ ]:
# referencia de lo que vamos a reimplementar
print 'sumEveryOther: \n{0}'.format(sumEveryOther)
print '\nsumEveryThird: \n{0}'.format(sumEveryThird)

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
# usa np.tile y np.eye para reconstruir los arrays
sumEveryOtherTile = <RELLENA>
sumEveryThirdTile = <RELLENA>

print sumEveryOtherTile
print 'sumEveryOtherTile.dot(vector): {0}'.format(sumEveryOtherTile.dot(vector))
print '\n', sumEveryThirdTile
print 'sumEveryThirdTile.dot(vector): {0}'.format(sumEveryThirdTile.dot(vector))

In [ ]:
# TEST reconstrucción con `np.tile` y `np.eye` (4b)
Test.assertEquals(sumEveryOtherTile.shape, (2, 6), 'forma incorrecta sumEveryOtherTile')
Test.assertEquals(sumEveryThirdTile.shape, (3, 6), 'forma incorrecta sumEveryThirdTile')
Test.assertTrue(np.allclose(sumEveryOtherTile.dot(vector), [6, 9]),
                'valor incorrecto sumEveryOtherTile')
Test.assertTrue(np.allclose(sumEveryThirdTile.dot(vector), [3, 5, 7]),
                'valor incorrecto sumEveryThirdTile')

#### **(4c) Reconstrucción con `np.kron` **
#### El producto Kronecker es la generalización del outer product sobre matrices, abajo tienes ejemplos para ilustrar la idea. En [Wikipedia](https://en.wikipedia.org/wiki/Kronecker_product) tienes una definición más detallada.  Podemos usar [np.kron](http://docs.scipy.org/doc/numpy/reference/generated/numpy.kron.html) para calcular los productos Kronecker y reconstruir los arrays `sumBy`.  $ \otimes $ indica un productor Kronecker.

#### $$ \begin{bmatrix} 1 & 2 \\\ 3 & 4 \end{bmatrix} \otimes \begin{bmatrix} 1 & 2 \end{bmatrix}  = \begin{bmatrix} 1 \cdot 1 & 1 \cdot 2 & 2 \cdot 1 & 2 \cdot 2 \\\ 3 \cdot 1 & 3 \cdot 2 & 4 \cdot 1 & 4 \cdot 2 \end{bmatrix} = \begin{bmatrix} 1 & 2 & 2 & 4 \\\ 3 & 6 & 4 & 8 \end{bmatrix}  $$
####  Podemos ver como el producto Kronecker continúa expandiendose si añadimos otra fila al segundo array.
#### $$ \begin{bmatrix} 1 & 2 \\\ 3 & 4 \end{bmatrix} \otimes \begin{bmatrix} 1 & 2 \\\ 3 & 4 \end{bmatrix} = \begin{bmatrix} 1 \cdot 1 & 1 \cdot 2 & 2 \cdot 1 & 2 \cdot 2 \\\ 1 \cdot 3 & 1 \cdot 4 & 2 \cdot 3 & 2 \cdot 4 \\\ 3 \cdot 1 & 3 \cdot 2 & 4 \cdot 1 & 4 \cdot 2 \\\ 3 \cdot 3 & 3 \cdot 4 & 4 \cdot 3 & 4 \cdot 4 \end{bmatrix} = \begin{bmatrix} 1 & 2 & 2 & 4 \\\ 3 & 4 & 6 & 8 \\\ 3 & 6 & 4 & 8 \\\ 9 & 12 & 12 & 16 \end{bmatrix} $$
#### En este ejercicio reconstruirás los arrays `sumByThree` y `sumByTwo` usando `np.kron`, `np.eye`, y `np.ones`.   `np.ones` crea un array de unos.

In [ ]:
# como referencia
print 'sumByThree: \n{0}'.format(sumByThree)
print '\nsumByTwo: \n{0}'.format(sumByTwo)

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
# usa np.kron, np.eye, y np.ones para reconstruir los arrais
sumByThreeKron = <RELLENA>
sumByTwoKron = <RELLENA>

print sumByThreeKron
print 'sumByThreeKron.dot(vector): {0}'.format(sumByThreeKron.dot(vector))
print '\n', sumByTwoKron
print 'sumByTwoKron.dot(vector): {0}'.format(sumByTwoKron.dot(vector))

In [ ]:
# TEST Reconstrucción con `np.kron` (4c)
Test.assertEquals(sumByThreeKron.shape, (2, 6), 'forma incorrecta sumByThreeKron')
Test.assertEquals(sumByTwoKron.shape, (3, 6), 'forma incorrecta sumByTwoKron')
Test.assertTrue(np.allclose(sumByThreeKron.dot(vector),  [3, 12]),
                'valor incorrecto sumByThreeKron')
Test.assertTrue(np.allclose(sumByTwoKron.dot(vector), [1, 5, 9]),
                'valor incorrecto sumByTwoKron')

#### **(4d) Agregación por tiempo** como vimos en la parte (4a) sería interesante incorporar el conocimiento experimental en nuestro análisis. Para hacerlo estudiaremos primero los aspectos temporales de la respuesta neuronal agregando nuestras características a lo largo del tiempo. En otras palabras, queremos ver como diferentes píxeles (y las neuronas capturadas en esos píxels) reacionan durante los 20 segundos en los que se muestran nuevos patrones sin importar que tipo de patrón es. Por lo que en vez de trabajar con 240 características individuales vamos a agregarlas a 20 nuevas características donde la primera característica nueva captura la respuesta un segundo despues de que aparezca un patrón visual, la segunda característica nueva es la respuesta después de dos segundos y asi sucesivamente.
#### Podemos realizar esta agregación utilizando una operación de map. Primero construimos un array multidimensional $ \scriptsize \mathbf{T} $ que cuando se le aplique la operación punto con un vector de 240 dimensiones sume las componentes de cada segundo para cada una de las 12 exposiciones y devuelva un vector de 20 dimensiones. Este ejercicio es similar al (4b). Una vez creado el array multidimensional $ \scriptsize \mathbf{T} $, usa una operación `map` con el array y cada serie temporal para generar un dataset transformado. Después lo cachearemos y haremos una cuenta de la salida ya que vamos a utilizarlo más veces.

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
# crea un array multidimensiona para realizar la agregación
T = <RELLENA>

# Transforma scaledData usando T recuerda guardar las claves
timeData = scaledData.<RELLENA>

timeData.cache()
print timeData.count()
print timeData.first()

In [ ]:
# TEST Agregación pro tiempo (4d)
Test.assertEquals(T.shape, (20, 240), 'forma incorrecta para T')
timeDataFirst = timeData.values().first()
timeDataFifth = timeData.values().take(5)[4]
Test.assertEquals(timeData.count(), 46460, 'valor incorrecto timeData')
Test.assertEquals(timeDataFirst.size, 20, 'valor incorrecto timeData')
Test.assertEquals(timeData.keys().first(), (0, 0), 'valor incorrecto timeData')
Test.assertTrue(np.allclose(timeDataFirst[:2], [0.00802155, 0.00607693]),
                'valor incorrecto timeData')
Test.assertTrue(np.allclose(timeDataFifth[-2:],[-0.00636676, -0.0179427]),
                'valor incorrecto timeData')

#### **(4e) Obtener una representación compacta**
#### Ahora tenemos un dataset agregado por tiempo con  $\scriptsize n = 46460$ pixels y $\scriptsize d = 20$ características temporales agregadas y queremos usar PCA para encontrar una representación más compacta. Usa la función  `pca` del apartado (2a) para realicar PCA en estos datos con $\scriptsize k = 3$ para obterner como resultado un dataset con una dimensionalidad de 40.460 x 3. Como antes necesitarás extraer los valores de `timeData` ya que es un RDD de pares clave-valor.

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
componentsTime, timeScores, eigenvaluesTime = <RELLENA>

print 'componentsTime: (first five) \n{0}'.format(componentsTime[:5,:])
print ('\ntimeScores (first three): \n{0}'
       .format('\n'.join(map(str, timeScores.take(3)))))
print '\neigenvaluesTime: (first five) \n{0}'.format(eigenvaluesTime[:5])

In [ ]:
# TEST representación compacta (4e)
Test.assertEquals(componentsTime.shape, (20, 3), 'forma incorrecta componentsTime')
Test.assertTrue(np.allclose(np.abs(np.sum(componentsTime[:5, :])), 2.37299020),
                'valor incorrecto componentsTime')
Test.assertTrue(np.allclose(np.abs(np.sum(timeScores.take(3))), 0.0213119114),
                'valor incorrecto timeScores')
Test.assertTrue(np.allclose(np.sum(eigenvaluesTime[:5]), 0.844764792),
                'valor incorrecto eigenvaluesTime')

#### ** Visualización 9: Los dos mejores componentes en el tiempo **
#### Veamos la puntuación de los dos PC más altos como una imagen compuesta. Cuando preprocesamos agregando por tiempo y después realizamos PCA sólo estamos mirando la variabilidad relacionada con las dinámicas temporales, como resultado si las neuronas parecen similares (tienen colores similares) en la imagen resultante significa que sus respuestas pueden variar de manera similiar a lo largo del tiempo independientemente de la dirección de codificación. En la imagen de abajo podemos definir la línea media como la línea horizontal a lo largo del cerebro y podemos ver claramente patrones de actividad neuronal en diferentes partes del cerebro y en concreto podemos ver que las regiones a cada lado de la línea media son similares, lo que sugiere que las dinámicas temporales no difieren a lo largo de las dos partes del cerebro.

In [ ]:
scoresTime = np.vstack(timeScores.collect())
imageOneTime = scoresTime[:,0].reshape(230, 202).T
imageTwoTime = scoresTime[:,1].reshape(230, 202).T
brainmap = polarTransform(3, [imageOneTime, imageTwoTime])

# generar la parrilla y dibujar los datos
fig, ax = preparePlot(np.arange(0, 10, 1), np.arange(0, 10, 1), figsize=(9.0, 7.2), hideLabels=True)
ax.grid(False)
image = plt.imshow(brainmap,interpolation='nearest', aspect='auto')
pass

#### **(4f) Aggregar por dirección **
#### A continuación vamos a realizar un segundo tipo de agregación de características de modo que podamos estudiar aspectos específicos a la dirección de la respuesta neuronal. En otras palabra queremos ver como píxels diferentes (y las neuronas capturadas por esos pixels) reaccionan cuando a un pez cebra se le presentan 12 patrones específicos de dirección ignarando el aspecto temporal de la reacción. En vez de trabajar con 240 características individuales agregaremos las características originales en 12 nuevas características donde la primera característica nueva captura la respuesta del pixel medio al primer patrón específico de dirección, la segunda característica nueva es la respuesta al segundo patrón visual específico de dirección y así sucesivamente.
 
#### Como en la parte (4c) diseñaremos un array multidimensional $ \scriptsize \mathbf{D} $ que cuando sea multiplicado por un vector de 240 dimensioes sume los primeros 20 componentes después los segundos 20 componentes y así sucesivamente. Este ejercicio es muy similar a (4c). Crea primero $ \scriptsize \mathbf{D} $ después usa una operación `map` con ese array y cada serie temporal para generar un dataset transformado. Cachearemos y contaremos la salida ya que la usaremos más adelante.

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
# Crea un array multidimensional para realizar la agregación
D = <RELLENA>

# Transforma scaledData usando D, Transform scaledData using D. Recuerda mantener las claves.
directionData = scaledData.<RELLENA>

directionData.cache()
print directionData.count()
print directionData.first()

In [ ]:
# TEST agregar por dirección (4f)
Test.assertEquals(D.shape, (12, 240), 'forma incorrecta para D')
directionDataFirst = directionData.values().first()
directionDataFifth = directionData.values().take(5)[4]
Test.assertEquals(directionData.count(), 46460, 'longitud incorrecta de directionData')
Test.assertEquals(directionDataFirst.size, 12, 'valor de longitud incorrecta directionData')
Test.assertEquals(directionData.keys().first(), (0, 0), 'claves incorrectas en directionData')
Test.assertTrue(np.allclose(directionDataFirst[:2], [ 0.03346365,  0.03638058]),
                'valores incorrectos en directionData')
Test.assertTrue(np.allclose(directionDataFifth[:2], [ 0.01479147, -0.02090099]),
                'valores incorrectos en directionData')

#### **(4g) Representación compacta de los datos de dirección**
#### Ahora tenemos un dataset agregado por dirección con $\scriptsize n = 46460$ pixels y $\scriptsize d = 12$ características de dirección agregadas y queremos usar PCA para encontrar un modo de representación más compacto. Usa la función de `pca` del apartado (2a) para realizar PCA en estos datos con  $\scriptsize k = 3$, dando como resultado un dataset de 46.460 x 3. Como antes necesitarás extrer los valores de `directionData` ya que es un RDD de pares clave-valor.

In [ ]:
# TODO: Sustituye <RELLENA> con el código apropiado
componentsDirection, directionScores, eigenvaluesDirection = <RELLENA>

print 'componentsDirection: (first five) \n{0}'.format(componentsDirection[:5,:])
print ('\ndirectionScores (first three): \n{0}'
       .format('\n'.join(map(str, directionScores.take(3)))))
print '\neigenvaluesDirection: (first five) \n{0}'.format(eigenvaluesDirection[:5])

In [ ]:
# TEST representación compacta de datos de dirección (4g)
Test.assertEquals(componentsDirection.shape, (12, 3), 'forma incorrecta componentsDirection')
Test.assertTrue(np.allclose(np.abs(np.sum(componentsDirection[:5, :])), 1.080232069),
                'valor incorrecto componentsDirection')
Test.assertTrue(np.allclose(np.abs(np.sum(directionScores.take(3))), 0.10993162084),
                'valor incorrecto directionScores')
Test.assertTrue(np.allclose(np.sum(eigenvaluesDirection[:5]), 2.0089720377),
                'valor incorrecto eigenvaluesDirection')

 
#### **Visualización 10: Los dos mejores componentes por dirección**
#### Vamos a ver las puntuaciones de los dos PC más altos como una imagen compuesta. Cuando preprocesamos promediando a lo largo del tiempo (agrupando por dirección) y realizamos PCA solo estamos mirando la variabilidad relacionada con la dirección de los estímulos, como resultado si las neuronas parecen similares (tienen colores similares) en la imagen significa que sus respuestas varían de manera similiar a lo largo de las direcciones sin importar como evolucionan sobre el tiempo. En la imagen de abajo vemos un patrón diferente de similitud a lo largo de las regiones del cerebro. Es  más, las regiones a ambos lados de la línea media tienen colores diferentes lo que sugiere que estamos mirando a una propiedad (selección de la dirección) que tiene una representación  diferente en ambos lados del cerebro.

In [ ]:
scoresDirection = np.vstack(directionScores.collect())
imageOneDirection = scoresDirection[:,0].reshape(230, 202).T
imageTwoDirection = scoresDirection[:,1].reshape(230, 202).T
brainmap = polarTransform(2, [imageOneDirection, imageTwoDirection])
# with thunder: Colorize(cmap='polar', scale=2).transform([imageOneDirection, imageTwoDirection])

# generar parrila y trazar datos
fig, ax = preparePlot(np.arange(0, 10, 1), np.arange(0, 10, 1), figsize=(9.0, 7.2), hideLabels=True)
ax.grid(False)
image = plt.imshow(brainmap, interpolation='nearest', aspect='auto')
pass

#### **(4h) pasos siguientes**
#### En el análisis anterior hemos identificado con éxito regiones del cerebro que codifican  propiedades particulares como por ejemplo un patrón temporal particular o la reacción a un estímulo. Sin embargo esto sólo es el primer paso. A estos análisis exploratorios les siguen una investigación más profunda, que podría por ejemplo implicar el uso de un problema de regresión lineal.